In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('FakeNewstrain.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
df.shape

(20800, 5)

In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
df = df.fillna('')

In [10]:
df.shape

(20800, 5)

In [32]:
df['content'] = df['author'] + ' ' + df['title']

In [33]:
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [34]:
y = df['label']

In [35]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [36]:
X = df.drop(columns='label',axis=1)

In [37]:
X

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [38]:
ps = PorterStemmer()

In [39]:
def stemmimg(content):
    stem_content = re.sub('[^A-Za-z]',' ',content)
    stem_content = stem_content.lower()
    stem_content = stem_content.split()
    stem_content = [ps.stem(words) for words in stem_content if not words in stopwords.words('english')]
    stem_content = ' '.join(stem_content)
    return stem_content

In [40]:
df['content'] = df['content'].apply(stemmimg)

In [41]:
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [42]:
X = df['content'].values
Y = df['label'].values

In [43]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [44]:
print(Y)

[1 0 1 ... 0 1 1]


In [45]:
Y.shape

(20800,)

In [46]:
X.shape

(20800,)

In [47]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [48]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [51]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [56]:
lr = LogisticRegression()

In [58]:
lr.fit(X_train,y_train)

LogisticRegression()

In [60]:
X_train_pred = lr.predict(X_train)

In [61]:
X_train_pred

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [64]:
print('accuray_score =',accuracy_score(X_train_pred,y_train))

accuray_score = 0.9865985576923076


In [66]:
print('conf_matrix - \n',confusion_matrix(X_train_pred,y_train))

conf_matrix - 
 [[8136   49]
 [ 174 8281]]


In [67]:
print('classification_report - \n',classification_report(X_train_pred,y_train))

classification_report - 
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      8185
           1       0.99      0.98      0.99      8455

    accuracy                           0.99     16640
   macro avg       0.99      0.99      0.99     16640
weighted avg       0.99      0.99      0.99     16640



In [68]:
X_test_pred = lr.predict(X_test)

In [71]:
print('accuray_score =',accuracy_score(X_test_pred,y_test))
print('\n')
print('*'*75)
print('conf_matrix - \n',confusion_matrix(X_test_pred,y_test))
print('\n')
print('*'*75)
print('classification_report - \n',classification_report(X_test_pred,y_test))

accuray_score = 0.9790865384615385


***************************************************************************
conf_matrix - 
 [[2004   14]
 [  73 2069]]


***************************************************************************
classification_report - 
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      2018
           1       0.99      0.97      0.98      2142

    accuracy                           0.98      4160
   macro avg       0.98      0.98      0.98      4160
weighted avg       0.98      0.98      0.98      4160



In [76]:
X_new = X_test[1]
pred = lr.predict(X_new)
print(pred)

if (pred[0]==0):
    print('The news is real')
else:
    print('Its a fake news')


[0]
The news is real


In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [79]:
!pip3 install xgboost

In [87]:
import xgboost as xgb

In [81]:
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [82]:
X_train_rfc_pred = rfc.predict(X_train)

In [83]:
print('accuray_score =',accuracy_score(X_train_rfc_pred,y_train))
print('\n')
print('*'*75)
print('conf_matrix - \n',confusion_matrix(X_train_rfc_pred,y_train))
print('\n')
print('*'*75)
print('classification_report - \n',classification_report(X_train_rfc_pred,y_train))

accuray_score = 1.0


***************************************************************************
conf_matrix - 
 [[8310    0]
 [   0 8330]]


***************************************************************************
classification_report - 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8310
           1       1.00      1.00      1.00      8330

    accuracy                           1.00     16640
   macro avg       1.00      1.00      1.00     16640
weighted avg       1.00      1.00      1.00     16640



In [84]:
X_test_rfc_pred = rfc.predict(X_test)

In [85]:
print('accuray_score =',accuracy_score(X_test_rfc_pred,y_test))
print('\n')
print('*'*75)
print('conf_matrix - \n',confusion_matrix(X_test_rfc_pred,y_test))
print('\n')
print('*'*75)
print('classification_report - \n',classification_report(X_test_rfc_pred,y_test))

accuray_score = 0.9927884615384616


***************************************************************************
conf_matrix - 
 [[2058   11]
 [  19 2072]]


***************************************************************************
classification_report - 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      2069
           1       0.99      0.99      0.99      2091

    accuracy                           0.99      4160
   macro avg       0.99      0.99      0.99      4160
weighted avg       0.99      0.99      0.99      4160



In [89]:
modelxg = xgb.XGBClassifier()
modelxg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [90]:
X_train_xg_pred = modelxg.predict(X_train)

In [91]:
X_test_xg_pred = modelxg.predict(X_test)

In [92]:
print('accuray_score =',accuracy_score(X_test_xg_pred,y_test))
print('\n')
print('*'*75)
print('conf_matrix - \n',confusion_matrix(X_test_xg_pred,y_test))
print('\n')
print('*'*75)
print('classification_report - \n',classification_report(X_test_xg_pred,y_test))

accuray_score = 0.9889423076923077


***************************************************************************
conf_matrix - 
 [[2045   14]
 [  32 2069]]


***************************************************************************
classification_report - 
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      2059
           1       0.99      0.98      0.99      2101

    accuracy                           0.99      4160
   macro avg       0.99      0.99      0.99      4160
weighted avg       0.99      0.99      0.99      4160



In [93]:
print('accuray_score =',accuracy_score(X_train_xg_pred,y_train))
print('\n')
print('*'*75)
print('conf_matrix - \n',confusion_matrix(X_train_xg_pred,y_train))
print('\n')
print('*'*75)
print('classification_report - \n',classification_report(X_train_xg_pred,y_train))

accuray_score = 0.9944110576923076


***************************************************************************
conf_matrix - 
 [[8229   12]
 [  81 8318]]


***************************************************************************
classification_report - 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      8241
           1       1.00      0.99      0.99      8399

    accuracy                           0.99     16640
   macro avg       0.99      0.99      0.99     16640
weighted avg       0.99      0.99      0.99     16640

